# 1) Mean and Std

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from BSDE import *
import warnings 
import bokeh
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show, output_file
import time
warnings.filterwarnings("ignore")

### Fixed param

In [3]:
T = 0.25
K = 100.
r = 0.01
R = 0.06
p = 20
M = np.eye(p)
S0 = 100.
mu = 0.05
sigma = 0.2
Q = 0

### Param for Mesh run 

In [8]:
m = 8
N = 3000
p = 20
n_picard = 3
M_run = 20

In [ ]:
a = np.zeros(M_run)
timing = np.zeros_like(a)
for i in range (M_run): 
    start = time.time()
    test = BsdeHD(T, K, M, mu, Q, sigma, S0, r, R)
    a[i] = test.get_price_mesh(N, m, n_picard=n_picard, option_type='call combination')
    elapsed = time.time() - start
    timing[i] = round(elapsed, 3)

In [12]:
a.mean()

8.4943561135790642

In [8]:
import pandas as pd

In [9]:
param = ['q', 'mu', 'R ', 'r ','sigma', 'T', 'K', 'S0']
values = [q, mu, R, r, sigma, T, K, S0]
dataset = list(zip(param,values))
df_param = pd.DataFrame(data = dataset, columns=['Parameters', 'values'])

In [10]:
sparam = ['N_particles', 'm_discretization', 'N_run', 'n_picard']
svalues = [N, m, M_run, n_picard]
sp_dataset = list(zip(sparam,svalues))
df_sparam = pd.DataFrame(sp_dataset, columns=['Specific Parameters', 'values'])
df_sparam

,Specific Parameters,values
0,N_particles,4000
1,m_discretization,12
2,N_run,20
3,n_picard,10


In [11]:
run_id = ["run {}".format(i) for i in range(1, M_run+1)]
df = pd.DataFrame(list(zip(run_id, a, timing)), columns = ['run','values', 'time'])

In [12]:
min_a = round(df['values'].min(), 4)
max_a = round(df['values'].max(), 4)
mean_a = round(df['values'].mean(), 4)
std_a = round(df['values'].std(), 4)
left = round(mean_a - 1.96 * std_a / M_run, 4)
right = round(mean_a + 1.96 * std_a / M_run, 4)

In [13]:
stat_param = ['mean', 'std', '95% confidence interval', 'min', 'max']
stat_values = [mean_a, std_a, [left, right], min_a, max_a]
stat_data = list(zip(stat_param, stat_values))
stat_df = pd.DataFrame(stat_data, columns = ['stat parameter', 'values'])
stat_df

,stat parameter,values
0,mean,7.1875
1,std,0.1387
2,95% confidence interval,"[7.1739, 7.2011]"
3,min,6.9085
4,max,7.4492


In [14]:
df_join = pd.concat([df_param, df_sparam, df, stat_df], axis = 0)
df_join.to_csv('output_mesh.csv')